In [ ]:
'''A messy notebook for generating the embeddings from photos.'''

import pandas as pd
import torch 

from pathlib.Path import home
from pytorch_lightning import Trainer
from src.train import SphereClassifier, WhaleDataModule
from config.config import load_config

# config files - should reflect how the model was trained 
cfg = load_config('config/efficientnet_v2m.yaml', "config/default.yaml")

# parent directory for input data
INPUT_DIR = f'{home}/images_to_test'     

# subdirectory with images
image_dir = f"{INPUT_DIR}/images"                      

# you'll need to have a dummy csv with one column "images" with the image file names
LABEL_DF = pd.read_csv(f'{INPUT_DIR}/test_image_labels.csv') 

# parent directory for output data
#   note that an output dir like this is created by training the model, e.g.,
#   python -m src.train --config_path config/efficientnet_b6.yaml --exp_name b6
OUT_DIR = "../checkpoints/" # 'result/v2m/-1' 
OUT_PATH = f'{OUT_DIR}/test_embeddings.npz' # name of output path 

# create data module
data_module = WhaleDataModule(
    LABEL_DF, 
    cfg, 
    image_dir=image_dir, 
    val_bbox_name="none", # used if you need to add bounding boxes around animals, see src.dataset.load_bbox()
    fold=-1               # -1 means use whole dataset
)

# build model and trainer 
model = SphereClassifier(cfg)

trainer = Trainer(gpus=torch.cuda.device_count(), sync_batchnorm=True, precision=16)

# load weights from checkpoint. if you trained model via src.train, should be a file like this in the OUT_DIR
check_path = f'{OUT_DIR}/last.ckpt'
model = model.load_from_checkpoint(check_path)

# define output file path
model.test_results_fp = OUT_PATH

# build and save feature vectors 
trainer.test(model, data_module.all_dataloader())